## **Komórki związane z pobraniem danych oraz wyborem danych dla wskazanych klas do dalszej analizy**

In [57]:
import numpy as np
import pandas as pd

#funkcja odpakowująca batch i zwracająca słownik
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [58]:
#odpakowanie danych treningowych (batchów 1-5)
batch1 = unpickle('data_batch_1');
batch2 = unpickle('data_batch_2');
batch3 = unpickle('data_batch_3');
batch4 = unpickle('data_batch_4');
batch5 = unpickle('data_batch_5');

#odpakowanie danych testowych
test_batch = unpickle('test_batch');

#odpakowanie pliku batches.meta
batches = unpickle('batches.meta');

In [59]:
#wyświetlenie nazw labeli określających klasy 
#- szukamy airplane, cat i truck
batches

{b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_cases_per_batch': 10000,
 b'num_vis': 3072}

In [60]:
#funkcja wyciągająca dane i labele ze słowników
def take_data_and_labels(dict):
  X = dict[b'data']
  Y = dict[b'labels']
  return X, Y

In [61]:
def all_classes(batch):
  X, Y = take_data_and_labels(batch)
  #zamiana na obiekty typu dataframe
  df = pd.DataFrame(X)
  df2 = pd.DataFrame(Y)
  #zmiana nazwy kolumny oznaczającej rodzaj klasy
  df2 = df2.rename(columns={0: 'label'})
  #połączenie X i Y w jeden dataframe
  dataset = pd.merge(df, df2, left_index=True, right_index=True, how='outer')
  #wyciągnięcie klas 0, 3 i 9
  dataset0 = dataset.loc[dataset['label'] == 0]
  dataset3 = dataset.loc[dataset['label'] == 3]
  dataset9 = dataset.loc[dataset['label'] == 9]
  #połączenie wszystkich klas w jeden obiekt typu dataframe
  dataset = dataset0.append(dataset3)
  dataset = dataset.append(dataset9)
  return dataset

In [62]:
#połączenie wszystkich batchy w jeden dataframe danych treningowych
batch1_df = all_classes(batch1)
batch2_df = all_classes(batch2)
batch3_df = all_classes(batch3)
batch4_df = all_classes(batch4)
batch5_df = all_classes(batch5)

train = batch1_df.append(batch2_df)
train = train.append(batch3_df)
train = train.append(batch4_df)
train = train.append(batch5_df)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
29,202,202,204,206,208,209,210,212,213,215,216,216,215,216,216,216,217,217,218,218,216,217,218,214,213,212,211,209,208,207,205,203,206,206,207,209,211,212,213,215,...,202,219,234,242,245,245,245,221,220,221,219,214,201,172,135,97,70,58,54,53,53,52,53,55,56,60,70,88,106,127,149,174,195,212,227,237,243,243,243,0
30,126,122,126,127,130,130,132,133,130,132,134,131,131,134,133,136,137,137,136,131,130,132,132,132,129,127,127,125,124,124,120,117,122,119,121,124,129,130,130,130,...,115,116,116,113,109,102,95,104,104,107,109,110,110,112,110,107,109,109,109,110,107,108,110,110,111,108,104,111,115,117,117,117,116,114,113,111,109,107,102,0
35,251,247,247,248,249,248,248,248,249,249,248,249,250,249,249,249,249,248,249,249,250,250,250,249,250,250,250,250,245,229,244,251,249,246,246,246,247,247,247,248,...,32,29,29,31,26,27,22,42,43,36,36,40,35,35,37,38,38,35,31,31,28,31,37,38,25,25,25,27,23,19,27,27,28,32,30,36,35,33,26,0
49,255,255,255,182,181,255,255,255,255,255,255,255,255,255,255,255,255,255,254,254,255,255,255,255,255,255,255,255,255,255,255,255,255,252,255,170,84,236,255,253,...,253,253,253,254,254,254,254,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,232,204,243,255,255,255,255,255,255,255,255,0
77,44,47,51,54,53,53,53,54,54,58,66,59,58,66,58,60,61,64,66,73,63,64,71,69,72,73,72,73,71,75,72,74,52,47,51,47,52,54,59,61,...,55,53,53,45,49,49,46,19,44,36,42,32,34,35,35,45,51,50,42,42,45,38,42,40,49,43,48,48,43,50,47,47,43,53,48,44,51,52,43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,183,178,180,182,185,187,192,197,199,201,201,202,205,206,209,212,214,214,215,216,216,217,217,216,219,221,222,225,226,230,232,234,180,175,187,189,190,191,196,203,...,70,70,71,71,71,70,69,98,96,98,98,95,92,91,87,81,82,86,86,80,75,73,73,72,76,76,73,72,74,74,75,76,74,75,77,76,75,75,77,9
9945,255,255,255,253,253,252,251,253,253,253,254,255,254,255,255,255,254,251,244,232,223,212,202,190,180,193,216,238,249,255,254,255,255,255,255,254,253,255,254,254,...,255,255,255,255,255,255,255,254,233,123,86,92,166,228,238,255,255,254,254,254,254,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,9
9963,147,150,138,122,158,152,98,68,85,103,108,90,86,120,172,171,154,169,166,159,165,151,115,105,100,104,101,87,82,82,121,138,157,156,125,129,150,141,121,103,...,92,87,85,85,89,96,98,59,47,51,50,48,48,37,37,57,80,78,41,19,39,48,90,96,90,87,88,92,96,102,105,105,100,97,93,89,89,88,90,9
9971,229,238,240,253,180,83,124,167,202,227,243,240,247,255,255,238,164,230,227,215,214,226,235,245,254,254,255,255,255,255,254,254,239,238,242,225,77,12,45,85,...,120,116,119,120,121,123,124,131,133,136,127,117,108,111,118,128,128,124,121,120,123,127,128,129,126,122,124,126,126,129,129,128,127,126,125,123,124,126,127,9


In [63]:
#dane testowe
test = all_classes(test_batch)
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
3,155,167,176,190,177,166,168,166,170,179,187,187,187,187,184,184,182,180,184,186,187,187,188,189,187,187,188,195,201,201,202,192,153,163,171,187,179,155,154,159,...,69,107,127,91,58,54,51,64,51,51,46,51,56,57,53,52,51,50,62,64,62,60,57,60,62,64,64,64,53,52,55,70,103,105,72,53,50,52,50,0
10,90,90,91,91,93,94,94,95,100,103,104,106,107,109,109,108,108,107,105,113,118,112,109,113,112,111,111,108,107,103,100,100,92,93,94,94,94,96,96,96,...,153,151,151,151,149,146,147,152,151,153,154,155,155,156,156,151,146,148,154,156,152,151,154,156,157,156,156,156,156,156,155,153,151,150,150,149,148,146,146,0
21,247,243,243,244,246,246,246,245,243,242,242,243,242,242,242,242,242,242,242,242,242,242,242,243,242,242,242,242,243,243,242,247,247,243,243,243,244,244,245,245,...,239,239,239,239,239,238,242,241,238,238,238,239,238,239,239,239,239,238,238,239,240,240,239,239,239,238,239,239,239,239,239,240,240,240,239,239,239,237,242,0
27,211,220,223,222,223,223,223,223,224,225,225,225,225,225,225,225,225,224,222,222,222,221,221,222,223,217,209,208,199,198,187,178,212,220,226,226,225,225,225,225,...,107,82,48,29,24,26,21,56,57,60,60,60,63,67,74,83,89,93,98,99,104,108,110,113,115,116,117,118,118,121,123,118,116,111,95,61,32,24,19,0
44,47,46,45,45,46,47,47,47,47,47,47,48,48,48,48,47,48,48,48,48,48,48,48,48,48,48,48,50,50,49,49,50,50,50,49,49,50,50,50,50,...,65,66,64,61,57,57,60,69,70,69,70,71,70,69,68,68,66,64,64,64,62,60,63,63,68,71,72,71,72,75,76,74,75,76,76,73,66,64,65,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,98,97,97,98,98,98,99,98,98,98,98,97,96,96,94,93,93,92,91,91,90,89,88,86,85,84,82,80,78,76,74,73,101,101,102,103,103,103,103,103,...,229,227,226,226,224,222,221,242,242,245,246,248,249,250,251,251,251,251,251,251,250,249,248,248,247,245,243,241,238,237,236,235,233,230,228,227,225,225,224,9
9927,244,241,242,243,244,244,244,243,242,242,242,241,237,236,237,237,237,237,237,237,238,240,241,240,239,240,239,238,237,236,236,235,244,241,243,243,243,243,242,242,...,184,184,184,184,186,184,184,185,186,187,185,181,181,180,182,185,185,186,183,179,180,181,184,188,189,190,191,188,184,184,184,184,186,188,187,188,190,189,187,9
9938,226,228,229,232,232,233,235,236,238,234,226,225,230,230,235,236,237,236,237,235,231,230,229,229,231,234,237,240,240,241,238,232,200,200,207,221,218,216,215,212,...,65,85,66,82,87,103,119,130,126,121,115,113,90,87,68,65,45,42,91,136,145,106,93,98,108,102,95,95,78,74,75,80,78,86,84,103,118,110,124,9
9958,56,31,38,73,122,188,157,152,196,208,225,246,241,185,123,162,230,250,250,234,174,169,195,229,212,232,252,253,249,245,245,244,59,40,39,73,93,92,70,89,...,158,157,181,186,183,135,106,154,128,80,42,22,10,5,5,5,3,0,2,6,15,17,4,0,3,38,78,100,127,172,170,170,163,161,174,182,162,66,51,9


In [64]:
#przetasowanie danych testowych i treningowych
train = train.sample(frac=1)
test = test.sample(frac=1)

In [65]:
#rozdzielenie danych testowych i treningowych na X i y
X_train = train.iloc[:, train.columns != 'label']
y_train = train[['label']]

X_test = test.iloc[:, test.columns != 'label']
y_test = test[['label']]

# **Komórki związane z przetwarzaniem wstępnym**



In [66]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 30)
pca = pca.fit(X_train)
X_train = pca.transform(X_train)

# **Komórki związane z klasyfikacją danych**

In [68]:
from sklearn.svm import SVC

model = SVC(kernel='rbf')
clf = model.fit(X_train, y_train.values.ravel())
X_test_new = pca.transform(X_test)
y_pred = clf.predict(X_test_new)

# **Komórki związane z walidacją modelu (dla danych testowych) wraz z wyznaczeniem i wydrukiem macierzy pomyłek oraz wartości F1**

In [69]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

print("Dokładność: ", accuracy_score(y_test, y_pred))
print("Macierz pomyłek: ")
print(confusion_matrix(y_test, y_pred))
print("F1: ", f1_score(y_test, y_pred, average='weighted'))

Dokładność:  0.8036666666666666
Macierz pomyłek: 
[[799 102  99]
 [ 76 824 100]
 [ 93 119 788]]
F1:  0.803671783799726


# **Komórka przygotowana do testowania z dodatkowym zbiorem testowym**

In [70]:
X_test = pca.transform(X_test)
y_pred = clf.predict(X_test)

print("Dokładność: ", accuracy_score(y_test, y_pred))
print("Macierz pomyłek: ")
print(confusion_matrix(y_test, y_pred))
print("F1: ", f1_score(y_test, y_pred, average='weighted'))

Dokładność:  0.8036666666666666
Macierz pomyłek: 
[[799 102  99]
 [ 76 824 100]
 [ 93 119 788]]
F1:  0.803671783799726
